# 문자열 필드
- Milvus에서 VARCHAR은 문자열 데이터를 저장하는 데 사용되는 데이터 유형
- VARCHAR 필드를 정의할 때 다음과 같은 두 개의 매개변수는 필수
    - datatype을 DataType.VARCHAR로 설정
    - VARCHAR 필드에 저장할 수 있는 최대 바이트 수를 정의 max_length를 지정, 1 ~ 65,535가 범위

* Milvus는 VARCHAR 필드에 대해 null 값과 기본값을 지원, 이러한 기본 기능을 사용하려면 nullable을 True, default_value을 문자열 값으로 설정

- 참고자료: https://milvus.io/docs/ko/string.md

# VARCHAR 필드 추가
- varchar_field1: 최대 100바이트까지 저장, null 값 허용, 기본값 "Unknown"
- varchar_field2: 최대 200바이트까지 저장, null 값 허용, 기본값 x

* enable_dynamic_fields=True을 설정하면 Milvus에서는 미리 정의하지 않은 스칼라 필드를 삽입할 수 있음

-> 쿼리 및 관리의 복잡성이 증가하여 성능에 영향을 미칠 수 있음

In [14]:
from pymilvus import MilvusClient, DataType

# Define server address
SERVER_ADDR = "http://localhost:19530"

# Create a MilvusClient instance
client = MilvusClient(uri=SERVER_ADDR)

# Define the collection schema
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_fields=True,
)

# Add 'varchar_field1' that supports up null values with default value "Unknown"
schema.add_field(field_name="varchar_field1", datatype=DataType.VARCHAR, max_length=100, nullable=True, default_value="Unknown")
# Add 'varchar_field2' that supports null values without default value
schema.add_field(field_name="varchar_field2", datatype=DataType.VARCHAR, max_length=200, nullable=True)
schema.add_field(field_name="pk", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=3)

{'auto_id': False, 'description': '', 'fields': [{'name': 'varchar_field1', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'default_value': string_data: "Unknown"
, 'nullable': True}, {'name': 'varchar_field2', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 200}, 'nullable': True}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3}}], 'enable_dynamic_field': False}

# 인덱스 매개변수 설정

In [ ]:
# Set index params
index_params = client.prepare_index_params()

# Index "varchar_field1" with AUTOINDEX
index_params.add_index(
    field_name="varchar_field1",
    index_type="AUTOINDEX",
    index_name="varchar_index"
)

# Index "embedding" with AUTOINDEX and specify metric_type
index_params.add_index(
    field_name="embedding",
    index_type="AUTOINDEX", # Use automatic indexing to simplify complex index settings
    metric_type="COSINE" # Specify similarity metric type, options include L2, COSINE, or IP
)

# 컬렉션 만들기

In [16]:
if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

# 데이터 삽입
- 컬렉션 생성 후 스키마와 일치하는 엔티티를 삽입

In [17]:
data = [
    {"varchar_field1": "Product A", "varchar_field2": "High quality product", "pk": 1, "embedding": [0.1, 0.2, 0.3]},
    {"varchar_field1": "Product B", "pk": 2, "embedding": [0.4, 0.5, 0.6]}, # varchar_field2 field is missing, which should be NULL
    {"varchar_field1": None, "varchar_field2": None, "pk": 3, "embedding": [0.2, 0.3, 0.1]},  # `varchar_field1` should default to `Unknown`, `varchar_field2` is NULL
    {"varchar_field1": "Product C", "varchar_field2": None, "pk": 4, "embedding": [0.5, 0.7, 0.2]},  # `varchar_field2` is NULL
    {"varchar_field1": None, "varchar_field2": "Exclusive deal", "pk": 5, "embedding": [0.6, 0.4, 0.8]},  # `varchar_field1` should default to `Unknown`
    {"varchar_field1": "Unknown", "varchar_field2": None, "pk": 6, "embedding": [0.8, 0.5, 0.3]},  # `varchar_field2` is NULL
    {"varchar_field1": "", "varchar_field2": "Best seller", "pk": 7, "embedding": [0.8, 0.5, 0.3]}, # Empty string is not treated as NULL
]

client.insert(
    collection_name="my_collection",
    data=data
)


{'insert_count': 7, 'ids': [1, 2, 3, 4, 5, 6, 7], 'cost': 0}

# 필터 표현식을 사용하여 쿼리
- 엔티티를 삽입 후 query 메서드를 사용하여 지정된 필터 표현식과 일치하는 엔티티를 검색


In [19]:
# Filter 'varchar_field1' with value "Product A"
filter = 'varchar_field1 == "Product A"'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["varchar_field1", "varchar_field2"]
)

print(res)

data: ["{'varchar_field1': 'Product A', 'varchar_field2': 'High quality product', 'pk': 1}"]


In [20]:
# Filter entities where `varchar_field2` is null
filter = 'varchar_field2 is null'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["varchar_field1", "varchar_field2"]
)

print(res)

# Example output:
# data: [
#     "{'varchar_field1': 'Product B', 'varchar_field2': None, 'pk': 2}",
#     "{'varchar_field1': 'Unknown', 'varchar_field2': None, 'pk': 3}",
#     "{'varchar_field1': 'Product C', 'varchar_field2': None, 'pk': 4}",
#     "{'varchar_field1': 'Unknown', 'varchar_field2': None, 'pk': 6}"
# ]


data: ["{'varchar_field2': None, 'varchar_field1': 'Product B', 'pk': 2}", "{'varchar_field2': None, 'varchar_field1': 'Unknown', 'pk': 3}", "{'varchar_field2': None, 'varchar_field1': 'Product C', 'pk': 4}", "{'varchar_field2': None, 'varchar_field1': 'Unknown', 'pk': 6}"]


In [21]:
# Filter entities with `varchar_field1` with value `Unknown`
filter = 'varchar_field1 == "Unknown"'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["varchar_field1", "varchar_field2"]
)

print(res)

# Example output:
# data: [
#     "{'varchar_field1': 'Unknown', 'varchar_field2': None, 'pk': 3}",
#     "{'varchar_field1': 'Unknown', 'varchar_field2': 'Exclusive deal', 'pk': 5}",
#     "{'varchar_field1': 'Unknown', 'varchar_field2': None, 'pk': 6}"
# ]


data: ["{'varchar_field1': 'Unknown', 'varchar_field2': None, 'pk': 3}", "{'varchar_field1': 'Unknown', 'varchar_field2': 'Exclusive deal', 'pk': 5}", "{'varchar_field1': 'Unknown', 'varchar_field2': None, 'pk': 6}"]


## 필터 표현식을 사용한 벡터 검색
- 기본 스칼라 필드 필터링 외에도 벡터 유사도 검색을 스칼라 필드 필터와 결합할 수 있음

In [23]:
# Search with string filtering

# Filter 'varchar_field2' with value "Best Seller"
filter = 'varchar_field2 == "Best seller"'

res = client.search(
    collection_name="my_collection",
    data=[[0.3, -0.6, 0.1]],
    limit=5,
    search_params={"params": {"nprobe": 10}},
    output_fields=["varchar_field1", "varchar_field2"],
    filter=filter
)

print(res)

data: [[{'pk': 7, 'distance': -0.04468163847923279, 'entity': {'varchar_field1': '', 'varchar_field2': 'Best seller'}}]]
